# Ensembale Mode here
Combine all the sub-model with Bagging method

In [32]:
import numpy as np
import pandas as pd
import scipy
import json
import seaborn as sns
from sklearn.base import TransformerMixin
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.pipeline import make_pipeline, Pipeline
import joblib
import matplotlib.pyplot as plt

### Figure the basic value

In [33]:
model_path = 'Choesn_model/'
configure_file_path = 'Choesn_model/Configuration.json'

TESTFILE = 'keyword.csv'
boolean = 'True'

## Define the MODEL object

In [37]:
class Model(object):

    def  __init__(self, model, name, test_set, topic, is_preprocess):
        self.model = model
        self.name = name
        self.test_set = test_set
        self.topic = topic
        self.is_preprocess = is_preprocess

    def show(self):
        print(
            self.name,
            self.test_set,
            self.topic,
            self.is_preprocess
        )


## Load the model detail from json figuration file

In [38]:
def load_configuration_to_model(file_path):
    with open(configure_file_path, 'r') as json_fp:
        configuration = json.load(json_fp)

    model_list = list()
    for model_figure in configuration:
        # read the figure
        model_file = joblib.load(model_path + model_figure['model_name'])
        name = model_figure['model_name']
        test_set = model_figure['test_set']
        topic = model_figure['topic']
        is_preprocess = boolean == model_figure['preprocess']

        # load to model list
        model = Model(model_file, name, test_set, topic, is_preprocess)
        model_list.append(model)

    return model_list

## Prepare the testing data and preprocess vector

In [39]:
model_list = load_configuration_to_model(configure_file_path)
for model in model_list:
    model.show()

tfidf_logi_pipe.joblib key_word_50 ['ARTS CULTURE ENTERTAINMENT', 'DOMESTIC MARKETS', 'SPORTS', 'SHARE LISTINGS', 'FOREX MARKETS', 'DEFENCE'] False
clf_NB_1.joblib key_word_50 ['SHARE LISTING', 'DOMESTIC MARKETS', 'ARTS CULTURE ENTERTAINMENT', 'SPORTS', 'DEFENCE', 'FOREX MARKETS'] False
clf_NB_2.joblib key_word_100 ['SHARE LISTING', 'DOMESTIC MARKETS', 'SPORTS', 'HEALTH', 'MONEY', 'DEFENCE'] False
clf_NB_3.joblib key_word_50 ['BIOGRAPHIES PERSONALITIES PEOPLE'] False
svm_proba_1.joblib key_word_100 ['DOMESTIC MARKETS', 'SHARE LISTINGS', 'ARTS CULTURE ENTERTAINMENT', 'SPORTS', 'DEFENCE', 'FOREX MARKETS', 'BIOGRAPHIES PERSONALITIES PEOPLE'] True
svm_proba_2.joblib article_words ['DOMESTIC MARKETS', 'SPORTS', 'FOREX MARKETS', 'HEALTH'] True
1_article_words.model article_words ['SPORTS', 'IRRELEVANT'] False
1_key_word_100.model key_word_100 ['SPORTS', 'IRRELEVANT'] False
2_article_words.model article_words ['SPORTS', 'IRRELEVANT'] False
